In [1]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [6]:
x = np.linspace(0, 2*np.pi, 2000)
print(x)
y = np.sin(x)

[0.00000000e+00 3.14316424e-03 6.28632847e-03 ... 6.27689898e+00
 6.28004214e+00 6.28318531e+00]


In [3]:
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5))
r = p.line(x, y, color="#2222aa", line_width=3)

In [4]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [5]:
show(p)

In [7]:
interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))

/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/notebook.py:256: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  warn("Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()")


<function __main__.update(f, w=1, A=1, phi=0)>

In [8]:
from __future__ import print_function, division

from timeit import default_timer as timer

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GlyphRenderer, LinearColorMapper
from bokeh.io import push_notebook
from numba import jit, njit

from ipywidgets import interact
import numpy as np
import scipy.misc

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
# smaller image
img_blur = (scipy.misc.ascent()[::-1,:]/255.0)[:250, :250].copy(order='C')
palette = ['#%02x%02x%02x' %(i,i,i) for i in range(256)]
width, height = img_blur.shape
p_blur = figure(x_range=(0, width), y_range=(0, height))
r_blur = p_blur.image(image=[img_blur], x=[0], y=[0], dw=[width], dh=[height], palette=palette, name='blur')

In [11]:
@njit
def blur(outimg, img, amt):
    iw, ih = img.shape
    for i in range(amt, iw-amt):
        for j in range(amt, ih-amt):
            px = 0.
            for w in range(-amt//2, amt//2):
                for h in range(-amt//2, amt//2):
                    px += img[i+w, j+h]
            outimg[i, j]= px/(amt*amt)

In [12]:
def update(i=0):
    level = 2*i + 1
    
    out = img_blur.copy()
    
    ts = timer()
    blur(out, img_blur, level)
    te = timer()
    print('blur takes:', te - ts)
    
    renderer = p_blur.select(dict(name="blur", type=GlyphRenderer))
    r_blur.data_source.data['image'] = [out]
    push_notebook(handle=t_blur)

In [13]:
t_blur = show(p_blur)

In [14]:
interact(update, i=(0, 10))

blur takes: 1.0517334509999614


/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/notebook.py:256: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  warn("Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()")


<function __main__.update(i=0)>

In [15]:
@jit
def getitem(img, x, y):
    w, h = img.shape
    if x >= w:
        x = w - 1 - (x - w)
    if y >= h:
        y = h - 1 - (y - h)
    return img[x, y]
 
def filter_factory(kernel):
    ksum = np.sum(kernel)
    if ksum == 0:
        ksum = 1
    k9 = kernel / ksum
 
    @jit
    def kernel_apply(img, out, x, y):
        tmp = 0
        for i in range(3):
            for j in range(3):
                tmp += img[x+i-1, y+j-1] * k9[i, j]
        out[x, y] = tmp
 
    @jit
    def kernel_apply_edge(img, out, x, y):
        tmp = 0
        for i in range(3):
            for j in range(3):
                tmp += getitem(img, x+i-1, y+j-1) * k9[i, j]
        out[x, y] = tmp
 
    @jit
    def kernel_k9(img, out):
        # Loop through all internals
        for x in range(1, img.shape[0] -1):
            for y in range(1, img.shape[1] -1):
                kernel_apply(img, out, x, y)
 
        # Loop through all the edges
        for x in range(img.shape[0]):
            kernel_apply_edge(img, out, x, 0)
            kernel_apply_edge(img, out, x, img.shape[1] - 1)
 
        for y in range(img.shape[1]):
            kernel_apply_edge(img, out, 0, y)
            kernel_apply_edge(img, out, img.shape[0] - 1, y)
 
    return kernel_k9

In [16]:
average = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
], dtype=np.float32)

sharpen = np.array([
    [-1, -1, -1],
    [-1, 12, -1],
    [-1, -1, -1],
], dtype=np.float32)

edge = np.array([
    [ 0, -1,  0],
    [-1,  4, -1],
    [ 0, -1,  0],
], dtype=np.float32)

edge_h = np.array([
    [ 0,  0,  0],
    [-1,  2, -1],
    [ 0,  0,  0],
], dtype=np.float32)

edge_v = np.array([
    [0, -1, 0],
    [0,  2, 0],
    [0, -1, 0],
], dtype=np.float32)

gradient_h = np.array([
    [-1, -1, -1],
    [ 0,  0,  0],
    [ 1,  1,  1],
], dtype=np.float32)

gradient_v = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1],
], dtype=np.float32)

sobol_h = np.array([
    [ 1,  2,  1],
    [ 0,  0,  0],
    [-1, -2, -1],
], dtype=np.float32)

sobol_v = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1],
], dtype=np.float32)
 
emboss = np.array([    
    [-2, -1, 0],
    [-1,  1, 1],
    [ 0,  1, 2],
], dtype=np.float32)

In [17]:
kernels = {
    "average"               : filter_factory(average),
    "sharpen"               : filter_factory(sharpen),
    "edge (both)"           : filter_factory(edge),
    "edge (horizontal)"     : filter_factory(edge_h),
    "edge (vertical)"       : filter_factory(edge_v),
    "gradient (horizontal)" : filter_factory(gradient_h),
    "gradient (vertical)"   : filter_factory(gradient_v),
    "sobol (horizontal)"    : filter_factory(sobol_h),
    "sobol (vertical)"      : filter_factory(sobol_v),
    "emboss"                : filter_factory(emboss),
}

In [18]:
images = {
    "ascent" : np.copy(scipy.misc.ascent().astype(np.float32)[::-1, :]),
    "face"   : np.copy(scipy.misc.face(gray=True).astype(np.float32)[::-1, :]),
}

In [19]:
palette = ['#%02x%02x%02x' %(i,i,i) for i in range(256)]
cm = LinearColorMapper(palette=palette, low=0, high=256)
width, height = images['ascent'].shape
p_kernel = figure(x_range=(0, width), y_range=(0, height))
r_kernel = p_kernel.image(image=[images['ascent']], x=[0], y=[0], dw=[width], dh=[height], color_mapper=cm, name="kernel")

In [20]:
def update(image="ascent", kernel_name="none", scale=100, bias=0):
    global _last_kname
    global _last_out
    
    img_kernel = images.get(image)

    kernel = kernels.get(kernel_name, None)
    if kernel == None:
        out = np.copy(img_kernel)

    else:
        out = np.zeros_like(img_kernel)

        ts = timer()
        kernel(img_kernel, out)
        te = timer()
        print('kernel takes:', te - ts)

    out *= scale / 100.0
    out += bias

    r_kernel.data_source.data['image'] = [out]
    push_notebook(handle=t_kernel)

In [21]:
t_kernel = show(p_kernel)